#### Name: Neel Shah
#### Mis no : 752462009
#### Batch no : 2


In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import models, layers
import joblib

### Fetch historical stock data

In [2]:
def fetch_stock_data(ticker, period="1y"):
    stock_data = yf.Ticker(ticker).history(period=period)
    stock_data['PriceChange'] = stock_data['Close'].pct_change().shift(-1)  # Percentage change
    stock_data['Target'] = (stock_data['PriceChange'] > 0).astype(int)  # Binary classification
    stock_data = stock_data.dropna()
    return stock_data

# Fetch data for training
print("Fetching data...")
data = fetch_stock_data("AAPL", period="1y")  # Apple stock as an example

# Features and target
X = data[['Open', 'High', 'Low', 'Close', 'Volume']]
y = data['Target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Fetching data...


### Define model

In [3]:
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Train model

In [4]:
print("Training model...")
history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2)

Training model...
Epoch 1/20
5/5 [==============================] - 2s 99ms/step - loss: 0.7031 - accuracy: 0.4429 - val_loss: 0.7036 - val_accuracy: 0.5143
Epoch 2/20
5/5 [==============================] - 0s 17ms/step - loss: 0.7018 - accuracy: 0.4429 - val_loss: 0.7017 - val_accuracy: 0.4571
Epoch 3/20
5/5 [==============================] - 0s 17ms/step - loss: 0.6906 - accuracy: 0.5286 - val_loss: 0.6997 - val_accuracy: 0.5143
Epoch 4/20
5/5 [==============================] - 0s 16ms/step - loss: 0.6883 - accuracy: 0.5500 - val_loss: 0.6949 - val_accuracy: 0.5429
Epoch 5/20
5/5 [==============================] - 0s 17ms/step - loss: 0.6912 - accuracy: 0.5286 - val_loss: 0.6900 - val_accuracy: 0.6000
Epoch 6/20
5/5 [==============================] - 0s 20ms/step - loss: 0.6848 - accuracy: 0.5286 - val_loss: 0.6871 - val_accuracy: 0.5714
Epoch 7/20
5/5 [==============================] - 0s 19ms/step - loss: 0.6824 - accuracy: 0.6214 - val_loss: 0.6851 - val_accuracy: 0.5429
Epoch 8/2

### Evaluate model

In [5]:
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"\nTest Accuracy: {test_accuracy:.4f}")

3/3 [==============================] - 0s 5ms/step - loss: 0.7003 - accuracy: 0.5000

Test Accuracy: 0.5000


### Generate predictions

In [6]:
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

3/3 [==============================] - 0s 4ms/step


### Generate classification report

In [7]:
report = classification_report(y_test, y_pred, target_names=["Down", "Up"])
print("\nClassification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

        Down       0.38      0.24      0.30        33
          Up       0.55      0.70      0.61        43

    accuracy                           0.50        76
   macro avg       0.46      0.47      0.45        76
weighted avg       0.47      0.50      0.48        76



### Save model and scaler

In [8]:
model.save("stock_model.h5")
joblib.dump(scaler, "stock_scaler.save")
print("Model and scaler saved successfully.")

Model and scaler saved successfully.


C:\Users\Neel\anaconda3\envs\breast_cancer_env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
